In [3]:
using LinearAlgebra
using IterativeSolvers

# Symmetric positive definite matrix
n = 5
A = Symmetric(randn(n, n))
A += n * I

# Right-hand side
b = randn(n)

# Solve Ax = b using Conjugate Gradient
x, history = cg(A, b; abstol=1e-8, maxiter=100, log=true)

println("Solution x = ", x)
println("Residuals = ", history.data)


Solution x = [0.24151510186656056, 0.2859605256516444, 0.538593094993676, -0.09916096718721909, 0.16912000934013016]
Residuals = Dict{Symbol, Any}(:reltol => 1.4901161193847656e-8, :abstol => 1.0e-8, :resnorm => [1.1774233324684862, 0.33886607402215163, 0.10030087180139968, 0.021253827651669555, 8.992579617258227e-17])


In [11]:
# import the identity matrix
I_mat = Matrix{Float64}(I, 2, 2)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [6]:
using LinearAlgebra

# create a one dimensional array of random numbers
x = rand(10)

diff = zeros(length(x))
for i in 1:length(x)
    diff[i] = abs(x[i] - x[end])  # this will work as intended
end

In [7]:
diff

10-element Vector{Float64}:
 0.5433695712637484
 0.08802143835663323
 0.467949932910234
 0.08755455282303526
 0.19441652079596394
 0.10380590963512382
 0.3254442926868497
 0.0075988564832883965
 0.10284311135775148
 0.0

In [1]:
function select_corrections_ORTHO(ν, m, droptol, t_initial, V_initial)
    # Algorithm 2: Selection of corrections (ORTHO)

    # Choose 0 << η < 1;
    # The pseudocode implies η is a parameter. Let's set a default or assume it's passed.
    # A value like 0.5 is common for such conditions (half of the 'old' norm).
    η = 0.5 

    # Set n_b := 0; maxorth := 2;
    n_b = 0
    maxorth = 2

    # Assuming t is a vector and V is a matrix (or a vector of vectors)
    # The pseudocode implies t_initial is a starting vector for each iteration i.
    # Let's assume t_initial is a function or an array that provides t_i for each i.
    # For simplicity, we'll assume t_initial is a mutable vector that gets updated.
    # If t_initial is meant to be different for each `i`, you'd need to provide that logic.
    t = deepcopy(t_initial) # Make a mutable copy

    # V_initial is used in the orthogonalization. Let's assume it's a matrix or a collection of vectors.
    # The pseudocode uses v_j, so V_initial should be a collection of vectors v_1, ..., v_m.
    # It also uses v_m+n_b, implying V grows.
    V = [V_initial...] # Convert to a mutable array of vectors if V_initial is a matrix or similar

    accepted_ts = [] # To store the accepted t_i vectors
    accepted_vs = [] # To store the accepted v_m+n_b vectors (which are normalized t_i)

    # for i := 1 : ν
    for i in 1:ν
        k = 0
        
        # We need a fresh t_i for each iteration of the outer loop.
        # The pseudocode isn't explicit about how t_i is generated for each 'i'.
        # For demonstration, let's assume `t` is re-initialized or passed in for each `i`.
        # If `t` is meant to evolve across `i`, then `t = t_initial` should be outside the loop.
        # Given the "t_i := (I - v_j v_j^T)t_i" part, it strongly suggests `t_i` is specific to the iteration.
        # For this example, let's assume `t` is the vector currently being processed for the `i`-th iteration.
        # You'll need to adapt this based on how your `t_i` is generated for each `i`.
        
        current_t_i = deepcopy(t) # Placeholder: you'll need to replace this with how your t_i is generated for each 'i'

        repeat
            old = LinearAlgebra.norm(current_t_i)
            k += 1

            # Orthogonalization: t_i := (I - v_j v_j^T)t_i for j := 1,...,m+n_b;
            # Assuming v_j are column vectors. (v_j v_j^T) is an outer product.
            # I - v_j v_j^T is a projection matrix.
            
            # The list of vectors to orthogonalize against is V (which initially contains V_initial)
            # and then gets appended with new vectors `v_{m+n_b}`
            
            num_current_basis_vectors = m + n_b
            
            for j in 1:num_current_basis_vectors
                if j <= length(V) # Check if V has enough elements
                    v_j = V[j]
                    
                    # Ensure v_j is a column vector for outer product if it's not already
                    if size(v_j, 2) != 1 && ndims(v_j) == 1
                        v_j = reshape(v_j, :, 1)
                    end
                    
                    # Projection: P = I - v_j * v_j'
                    # t_i = P * t_i = t_i - (v_j * v_j') * t_i = t_i - v_j * (v_j' * t_i)
                    # This is the Gram-Schmidt orthogonalization step.
                    current_t_i = current_t_i - v_j * (v_j' * current_t_i)
                else
                    # This should ideally not happen if num_current_basis_vectors is managed correctly.
                    # It means m + n_b is larger than the actual size of V.
                    # This might indicate a misunderstanding of how V is structured.
                    @warn "Attempted to access v_j beyond the current size of V. Skipping orthogonalization for j=$j."
                end
            end
        
        # until k = maxorth or ||t_i|| > η * old;
        # Julia's `repeat...until` loop
        until k == maxorth || LinearAlgebra.norm(current_t_i) > η * old

        # if ||t_i|| > droptol then {t_i is accepted}
        if LinearAlgebra.norm(current_t_i) > droptol
            n_b += 1
            
            # t̂_n_b := t_i / ||t_i||; vm+n_b := t̂_n_b
            # Normalize t_i and store it as a new basis vector
            normalized_t_i = current_t_i / LinearAlgebra.norm(current_t_i)
            
            push!(accepted_ts, current_t_i) # Store the accepted (unnormalized) t_i if needed
            push!(V, normalized_t_i) # Add the normalized vector to our basis set V

            # For consistency with the pseudocode where vm+n_b is assigned, we effectively update V.
            # The pseudocode's "vm+n_b := t̂_n_b" implies V is being augmented.
        end
    end # endfor

    return V, accepted_ts, n_b # Return the updated basis, accepted vectors, and final count
end

# --- Example Usage (Illustrative) ---

using LinearAlgebra # For norm function

# Define some dummy initial values for demonstration
# In a real scenario, t_initial and V_initial would come from your specific problem.
# Let's assume vectors are 3-dimensional.

# Example: t_initial is a vector that changes or is generated per iteration `i`.
# For simplicity, let's make it a fixed vector that is copied at the start of each `i` loop.
# In a real application, you'd have a function `get_t_i(i)` or `t_initial_vectors[i]`.
initial_t_vector_example = [1.0, 2.0, 3.0] 

# Example: V_initial contains 'm' initial orthogonal vectors (or basis vectors)
m_example = 2
V_initial_example = [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]] # Two orthogonal vectors

ν_example = 5 # Number of outer iterations
droptol_example = 1e-6 # A small tolerance value

# Call the function
updated_V, accepted_ts_list, final_n_b = select_corrections_ORTHO(
    ν_example,
    m_example,
    droptol_example,
    initial_t_vector_example,
    V_initial_example
)

println("Final n_b: ", final_n_b)
println("Number of accepted t's: ", length(accepted_ts_list))
println("Number of vectors in updated V: ", length(updated_V))
println("First few vectors in updated V:")
for i in 1:min(3, length(updated_V))
    println(updated_V[i])
end

Base.Meta.ParseError: ParseError:
# Error @ /home/fmereto/Master_arbeit/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:78:15
        # Julia's `repeat...until` loop
        until k == maxorth || LinearAlgebra.norm(current_t_i) > η * old
#             └────────────────────────────────────────────┘ ── Expected `end`

In [2]:
y = [1, 2, 3, 4, 5]
println(y)

[1, 2, 3, 4, 5]


In [3]:
A = zeros(5, 5)
for i in 1:5
    for j in 1:5
        A[i, j] = i + j
    end
end

display(A)

for i in 1:size(A, 1)
    println("norm:", norm(A[i, :]))
    println("sum:", sum(A[i, :]))

end

5×5 Matrix{Float64}:
 2.0  3.0  4.0  5.0   6.0
 3.0  4.0  5.0  6.0   7.0
 4.0  5.0  6.0  7.0   8.0
 5.0  6.0  7.0  8.0   9.0
 6.0  7.0  8.0  9.0  10.0

UndefVarError: UndefVarError: `norm` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing LinearAlgebra in the current active module Main

In [4]:
using LinearAlgebra

norms = zeros(size(A, 2))

non_conv_indices = Int[]
for i = 1:size(A, 2)
    norms[i] = norm(A[:, i])
    if norms[i] >= 13
        push!(non_conv_indices, i)
    end
end
println("Indices of non-convergent columns: ", non_conv_indices)

red = size(A, 2) -2

t = zero(size(A, 1), red)  # correction vectors

for (i, idx) in enumerate(non_conv_indices)
    t[:, i] = A[:, i] / norms[i]
end

Indices of non-convergent columns: [3, 4, 5]


MethodError: MethodError: no method matching zero(::Int64, ::Int64)
The function `zero` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  zero(!Matched::Type{Union{}}, ::Any...)
   @ Base number.jl:310
  zero(::Number)
   @ Base number.jl:308
  zero(!Matched::Type{LibGit2.GitHash})
   @ LibGit2 ~/julia-1.11.4/share/julia/stdlib/v1.11/LibGit2/src/oid.jl:221
  ...


In [5]:
B = zeros(size(A, 1), size(A, 2) - 3)
println(size(B))

(5, 2)


In [6]:
println("Hello, Julia!")

Hello, Julia!


In [7]:
println(collect(1:1:10))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [8]:
for i in collect(1:3:30)
    println(i)
end

1
4
7
10
13
16
19
22
25
28


In [9]:
println(rand())


0.38217798244332446


In [10]:

function sparse_matrix(N::Int, factor::Int)
    A = zeros(Float64, N, N)
    for i in 1:N
        for j in 1:N
            if i == j
                A[i, j] = rand() * factor
            else
                if rand() < 0.05 # 5% chance to be non-zero off-diagonal --> Problemmm
                    A[i, j] = rand() / factor
                else
                    # Keep the off-diagonal elements as zero
                    A[i, j] = 0.0
                end
            end
        end
    end

    return Hermitian(A)
end

sparse_matrix (generic function with 1 method)

In [11]:
A = sparse_matrix(10, 50)

10×10 Hermitian{Float64, Matrix{Float64}}:
 40.3653       0.0      0.0        …   0.00271354   0.0         0.0
  0.0         31.6632   0.0            0.0          0.0         0.0
  0.0          0.0     21.9434         0.0          0.0163261   0.0
  0.0          0.0      0.0            0.0          0.0         0.0
  0.0          0.0      0.0            0.0          0.0         0.0
  0.00223808   0.0      0.0        …   0.0          0.0         0.0
  0.0          0.0      0.0            0.0136667    0.0         0.0
  0.00271354   0.0      0.0           45.9249       0.0         0.0
  0.0          0.0      0.0163261      0.0         33.1983      0.0
  0.0          0.0      0.0            0.0          0.0        35.759

In [12]:
diag(A)

10-element Vector{Float64}:
 40.36528080369829
 31.663180955112196
 21.943439909397767
 41.033633157478015
 18.3034661483998
 40.33965585798469
 11.181322411420712
 45.92486958732117
 33.19825747335109
 35.75900908196829

In [13]:
using Plots
using LinearAlgebra

N = 2000
factor = 10
γ_pp = sparse_matrix(N, factor)

# Replace small values with a threshold
for i = 1:N
    for j = 1:N
        if abs(γ_pp[i,j] <= 1e-6)
            γ_pp[i,j] = 1e-6
        end
    end
end

logabs_γ_pp = log10.(abs.(γ_pp))
heatmap(logabs_γ_pp,
        xlabel="b",
        ylabel="a",
        colorbar_title="color map",
        aspect_ratio=:equal,
        c = :ice,
        size = (800, 800))


ArgumentError: ArgumentError: Cannot set a non-diagonal index in a Hermitian matrix